In [1]:
!pip install kagglehub

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("devildyno/email-spam-or-not-classification")

print("Path to dataset files:", path)

/Users/ougi/Documents/Project/Phishing-Detection-Using-ML/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /Users/ougi/.cache/kagglehub/datasets/devildyno/email-spam-or-not-classification/versions/1


In [3]:
import pandas as pd
import os
os.listdir(path)
df = pd.read_csv(os.path.join(path, 'spam_dataset.csv'))
df = df.rename(columns={
    'message_content': 'text',
    'is_spam': 'label'
})

df.head(5)

,text,label
0,"Hello Lonnie,\n\nJust wanted to touch base reg...",0
1,"Congratulations, you've won a prize! Call us n...",1
2,You have been pre-approved for a credit card w...,1
3,"Limited time offer, act now! Only a few spots ...",1
4,Your loan has been approved! Transfer funds to...,1


In [4]:
df = df.dropna(subset=["label"])
df["label_id"] = df["label"].astype(int)

In [5]:
df.to_csv(os.path.join(os.getcwd(), '..', 'data', 'train.csv'), index=False)